In [1]:
import rasterio

tif_path =r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif"

with rasterio.open(tif_path) as src:
    print("NoData value:", src.nodata)

NoData value: nan


In [1]:
import rasterio
import numpy as np

tif_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif"

with rasterio.open(tif_path) as src:
    data = src.read(masked=True)  # masked=True replaces NoData with masked entries
    total_pixels = data.size
    missing_pixels = np.ma.count_masked(data)
    missing_percentage = (missing_pixels / total_pixels) * 100

print(f"Total pixels: {total_pixels:,}")
print(f"Missing pixels: {missing_pixels:,}")
print(f"Missing percentage: {missing_percentage:.2f}%")


Total pixels: 809,710,080
Missing pixels: 23,487,325
Missing percentage: 2.90%


In [ ]:
import rasterio
import numpy as np
from rasterio.windows import Window
from rasterio.fill import fillnodata
from pathlib import Path

# Input and output paths
in_tif = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif")
out_tif = in_tif.with_name(in_tif.stem + "_filled_fast.tif")

# Define block size (smaller = less memory, larger = faster but needs more RAM)
BLOCK_SIZE = 1024  

with rasterio.open(in_tif) as src:
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32, nodata=None, compress="LZW")

    with rasterio.open(out_tif, "w", **profile) as dst:
        for band in range(1, src.count + 1):
            print(f"Processing band {band}/{src.count}...")
            for ji, window in src.block_windows(band):  # Iterate block by block
                arr = src.read(band, window=window).astype("float32")
                nod = src.nodatavals[band - 1]

                if nod is not None:
                    arr[arr == nod] = np.nan
                mask = np.isnan(arr)

                if mask.any():  # Fill only if nodata exists
                    arr_filled = fillnodata(
                        arr, mask=mask, max_search_distance=50, smoothing_iterations=0
                    )
                else:
                    arr_filled = arr

                dst.write(arr_filled, indexes=band, window=window)

print("✅ Faster block-wise filled raster saved at:", out_tif)


Processing band 1/5...
